# Simple RNN (Elman cell)

Implementation of an Elman cell architecture on time series data

In [7]:
import torch
import numpy as np
from torch import nn

import warnings

warnings.filterwarnings("ignore")

In [8]:
# get sample data

import polars as pl
import yfinance as yf
import re

prices = yf.download("SPLG", start='2023-01-01', end='2024-01-01')

df = (
    pl
    .from_pandas(
        prices
        .reset_index()
    ).with_columns(
        pl.lit("SPLG").alias("Ticker")
    )
)

df.columns = [re.sub(r"[^\w\s]","",header.split(",")[0]) for header in df.columns]

df.head()

[*********************100%***********************]  1 of 1 completed


Date,Close,High,Low,Open,Volume,Ticker
datetime[ns],f64,f64,f64,f64,i64,str
2023-01-03 00:00:00,43.092529,43.718036,42.75572,43.487079,3688279,"""SPLG"""
2023-01-04 00:00:00,43.42934,43.660294,42.996297,43.342732,4335811,"""SPLG"""
2023-01-05 00:00:00,42.928936,43.198383,42.861574,43.188762,4449438,"""SPLG"""
2023-01-06 00:00:00,43.900875,44.035599,42.928937,43.284993,2160602,"""SPLG"""
2023-01-09 00:00:00,43.881622,44.54081,43.862379,44.151073,4251681,"""SPLG"""


# SPLG Simple RNN Class

In [9]:
class ElmanRNN(nn.Module):
    def __init__(self):
        super(ElmanRNN, self).__init__()

        self.w_x = nn.Parameter(
            torch.rand(1),
            requires_grad=True
        )
        self.w_h = nn.Parameter(
            torch.rand(1),
            requires_grad=True
        )
        self.b_h = nn.Parameter(
            torch.rand(1),
            requires_grad=True
        )

        self.w_y = nn.Parameter(
            torch.rand(1),
            requires_grad=True
        )
        self.b_y = nn.Parameter(
            torch.rand(1),
            requires_grad=True
        )

    def cell(self, x, h):
        new_h = torch.relu(self.w_x * x + self.w_h * h + self.b_h)
        y = self.w_y * new_h + self.b_y

        return y, new_h

    def forward(self, x, h = None):
        """
        Inputs:

        x = input data
        h = hidden state value from previous iteration (default to 0 if not applicable)
        """

        if h is None:
            h = torch.zeros(1, dtype=torch.float32)
        
        for entry in x.t(): #loop is used to ensure hidden states carry through iterations
            output, h = self.cell(entry, h) 

        return output


## Training

### Prep dataset

In [10]:
# Suppose we create a model with 4 lagged variables as entry

data = df.select(pl.col("Close").alias("y"))
data = data.with_columns(
    pl.col("y").shift(1).alias("x_1"),
    pl.col("y").shift(2).alias("x_2"),
    pl.col("y").shift(3).alias("x_3"),
    pl.col("y").shift(4).alias("x_4"),
)
data = data[4:]

In [11]:
x_train = torch.FloatTensor(data.select("x_1", "x_2", "x_3", "x_4").to_numpy())
y_train = torch.FloatTensor(data["y"].to_numpy())

print(x_train.shape)
print(y_train.shape)

torch.Size([246, 4])
torch.Size([246])


### Setup Training Loop

In [12]:
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

# Setup hyperparamters
batch_size = 50
epochs = 1000
learning_rate = 0.0001

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

# Setup model
model = ElmanRNN()

# Setup loss function and optimizer
criterion = nn.MSELoss() #there is no RMSE
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, epochs + 1):
    total_loss = 0
    model.train()
    for batch_id, (data, target) in enumerate(train_loader): #for each batch, also get the index of batch
        optimizer.zero_grad()
        output = model(data) #forward pass
        loss = criterion(output, target)
        loss.backward() #compute gradients
        optimizer.step() #update weights
        
        total_loss += loss.item()

        if batch_id % 100 == 0 and epoch % 100 == 0: #update on training iterations
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id * len(data), len(train_loader.dataset),
                100. * batch_id / len(train_loader), loss.item()))


Train Epoch: 100 [0/246 (0%)]	Loss: 51.627842
Train Epoch: 200 [0/246 (0%)]	Loss: 2.201392
Train Epoch: 300 [0/246 (0%)]	Loss: 0.392060
Train Epoch: 400 [0/246 (0%)]	Loss: 0.397276
Train Epoch: 500 [0/246 (0%)]	Loss: 0.397617
Train Epoch: 600 [0/246 (0%)]	Loss: 0.397616
Train Epoch: 700 [0/246 (0%)]	Loss: 0.397602
Train Epoch: 800 [0/246 (0%)]	Loss: 0.397597
Train Epoch: 900 [0/246 (0%)]	Loss: 0.397577
Train Epoch: 1000 [0/246 (0%)]	Loss: 0.397573
